# Intro to class


[word embeddings colah](http://colah.github.io/posts/2014-07-NLP-RNNs-Representations/)

# Word Embeddings

Cuando empezó esto de los word embeddings? Hace algo de tiempo, aquí por ejemplo. [Distributed Representations](http://repository.cmu.edu/cgi/viewcontent.cgi?article=2841&context=compsci)

![Word Embeddings](https://qph.fs.quoracdn.net/main-qimg-e8b83b14d7261d75754a92d0d3605e36 "Word Embeddings")

Los word embeddings son vectores densos que representan tokens, aunque el término es word, sería más correcto usar token embeddings.

Hay varias implementaciones, como CBOW o Skip-Gram (del famoso word2vec) o Global Vectors (GloVe). 

Són una solución al problema de dimensionalidad que comentamos el primer día de clase.

### Vale pero que son realmente los valores de los word embeddings?

Los valores de los word embeddings representan "clases", aunque es muy complicado estar 100% seguro de que clases son, sobretodo porque cada vez que entrenemos word embeddings, el modelo puede aprender unas u otras clases. Con lo cuál, hay que olvidar-se un poco del concepto de vector dónde la posición 1 siempre tendremos la clase 1, y en la posición 2 la clase 2.

![](https://cdn-images-1.medium.com/max/1600/1*mLrheV1nGz7XemDAVRcZ4A.png)

Pensad que cuando escogemos la dimensionalidad de estos, ya estamos fijando cuántas clases puede aprender, y además, cada dimensionalidad pueden ser una mezcla de clases. Así pues, y el objetivo de aprender word vectors, es el siguiente:

> *Rezamos para que palabras "similares" tengan vectores similares*. Es decir a representar palabras por su semantismo.

### Recordatorio:

In [0]:
import numpy as np
import io

Estas representaciones son simples features, pero podemos mejorar de alguna forma? Si quisieramos representar un documento con la ultima representación y ngrams por ejemplo, creariamos un array tal que asi:

## Problemas que arreglamos

Los shapes de las representaciones son muy indicativas del problema que queremos resolver. Es decir, si ahora en lugar de una frase, queremos representar un documento de 400 posiciones, y que tiene 20k features? Por cada documento generaremos arrays de shape=(400, 20000), lo cual se hace insostenible a medida de que hagamos crecer las features. 

Ahora que ya habeis visto la cantidad de datos necesarios para ciertos problemas como computer vision, texto no ha sido indiferente a la revolución del "big data". Aún así, los word embeddings NO son deep learning, pero si se usan en él. Sás las features más usadas para texto en deep learning.

### Semántica

Además muchos problemas de NLP se basan basicamente en la semantica de estos tokens, y hasta la fecha no hemos visitado demasiado la semantica de las palabras. Usando Word Embeddings podremos boostear nuestras features y consecuentemente nuestros outputs de los algoritmos. 

Haremos uns dummy implementación del CBOW y GloVe, pero podéis encontrar vectores ya entrenados, o algoritmos implementados de forma eficiente en librerías como gensim o spaCy. 

CBOW esta basado en redes neuronales, y GloVe en factorización de matríces.

<div align="center">
    ![cbow.png](https://adriancolyer.files.wordpress.com/2016/04/word2vec-cbow.png =500x)
</div>

### Load Data

In [0]:
!pip install tqdm
from tqdm import tqdm

!pip install -q pydot

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot


Using TensorFlow backend.


In [0]:
!pip install spacy
!python -m spacy download es_core_news_sm

import spacy

nlp = spacy.load('es_core_news_sm', disable= ['ner', 'parser'])

In [0]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving quijote.txt to quijote (1).txt
User uploaded file "quijote.txt" with length 2141457 bytes


In [0]:
# construir el corpus
quijote = uploaded['quijote.txt']
quijote = io.StringIO(quijote.decode('utf-8')).getvalue()

quijote = quijote.split('.')


### Dataset Preparation

In [0]:
len(quijote)

8210

In [0]:
quijote_docs = []
for doc in tqdm(nlp.pipe(quijote, batch_size=1000, n_threads=4)):
    quijote_docs.append(doc)



0it [00:00, ?it/s]

1it [00:04,  4.64s/it]

1001it [00:08, 122.66it/s]

2001it [00:12, 164.90it/s]

3001it [00:16, 180.64it/s]

4001it [00:20, 194.90it/s]

5001it [00:24, 203.89it/s]

6001it [00:28, 211.70it/s]

7001it [00:32, 217.96it/s]

8001it [00:32, 243.00it/s]

8210it [00:32, 249.22it/s]

### Generate Dataset

### Create Model

In [1]:
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda

from sklearn.metrics.pairwise import euclidean_distances

Using TensorFlow backend.


### Train

### Get word embeddings

### Test

## GloVe

El algoritmo que veremos se basa en la siguiente idea:

<div align="center">
![](https://image.slidesharecdn.com/analyticssummitnov082013final-131114141831-phpapp01/95/analytics-summit-2013-25-638.jpg?cb=1384439169 =300x)
</div>

Es una idea que ya hemos visto presentada en formatos diferentes. Cuando entrenamos el Language Modeling, al final, estabamos asignando mayores probabilidades a palabras o tokens que aparecían en un contexto concreto, sólo que en ese caso, el contexto era sólo del pasado.

Vamos a ver una implementación "sencilla" de GloVe. Como hemos comentado, está basada en dos cosas, la primera, en la factorización de matrices. No entraremos en detalles de que es la factorización de matrices, nos basta con saber que en el caso de GloVe, nuestro objetivo será, dado una matriz de co-ocurrencias, obtener dos matrices que multiplicadas reproduzcan la original.

![glove.png](https://cdn-images-1.medium.com/max/800/1*UNtsSilztKXjLG99VXxSQw.png)

La matriz lila, será nuestra matriz de co-ocurrencias. ¿Qué es una matriz de coocurrencias? Tan fácil como ir sumando, dado un contexto, esas palabras que van aparenciendo entre si. Aquí tenéis un ejemplo de su uso.

![](https://cdn-images-1.medium.com/max/1600/0*Yl7I7bH52zk8m_8R.)

Primera Cosa en la que tenemos que fijarnos. Que valor tienen words y context? Es el mismo valor?

Y si es así, que hacemos con las dos matrices que generamos?

#### Algoritmo

![](https://i.imgur.com/HCo2ZwE.png)


### Notación en GloVe

* v_main: the word vector for the main word in the co-occurrence

* v_context: the word vector for the context word in the co-occurrence

* b_main: bias scalar for main word

* b_context: bias scalar for context word

* gradsq_W_main: a vector storing the squared gradient history for the main word vector (for use in the AdaGrad update)

* gradsq_W_context: a vector gradient history for the context word vector

* gradsq_b_main: a scalar gradient history for the main word bias

* gradsq_b_context: a scalar gradient history for the context word bias

* cooccurrence: the Xij value for the co-occurrence pair, described at length above




In [0]:
from scipy.sparse import lil_matrix
from math import log
from functools import partial

In [0]:
def build_cooccur(vocab, corpus, vocab_size, window_size=10, min_count=5):
    return "Co-ocurrence Matrix"

In [0]:
def run_iter(vocab, data, learning_rate=0.05, x_max=100, alpha=0.75):

    global_cost = 0

    # We want to iterate over data randomly so as not to unintentionally
    # bias the word vector contents
    shuffle(data)

    for (v_main, v_context, b_main, b_context, gradsq_W_main, gradsq_W_context,
         gradsq_b_main, gradsq_b_context, cooccurrence) in data:
        
        #FORWARD PASS
        weight = "Programar esto"

        cost_inner = "y esto"

        cost = "calcular el coste"

        # Add weighted cost to the global cost tracker
        global_cost += 0.5 * cost

        #BACKWARD/OPTIMIZATION PASS. Fuera del contenido, así que no lo implementaremos.
        # El algoritmo de optimización se llama AdaGrad, también disponible en Keras
        grad_main = cost_inner * v_context
        grad_context = cost_inner * v_main

        # Compute gradients for bias terms
        grad_bias_main = cost_inner
        grad_bias_context = cost_inner

        # Now perform adaptive updates
        v_main -= (learning_rate * grad_main / np.sqrt(gradsq_W_main))
        v_context -= (learning_rate * grad_context / np.sqrt(gradsq_W_context))

        b_main -= (learning_rate * grad_bias_main / np.sqrt(gradsq_b_main))
        b_context -= (learning_rate * grad_bias_context / np.sqrt(
                gradsq_b_context))

        # Update squared gradient sums
        gradsq_W_main += np.square(grad_main)
        gradsq_W_context += np.square(grad_context)
        gradsq_b_main += grad_bias_main ** 2
        gradsq_b_context += grad_bias_context ** 2

    return global_cost

In [0]:
def train_glove(vocab, cooccurrences, iter_callback=None, vector_size=100,
                iterations=25, **kwargs):
    """
    Train GloVe vectors on the given generator `cooccurrences`, where
    each element is of the form
        (word_i_id, word_j_id, x_ij)
    where `x_ij` is a cooccurrence value $X_{ij}$ as presented in the
    matrix defined by `build_cooccur` and the Pennington et al. (2014)
    paper itself.
    If `iter_callback` is not `None`, the provided function will be
    called after each iteration with the learned `W` matrix so far.
    Keyword arguments are passed on to the iteration step function
    `run_iter`.
    Returns the computed word vector matrix `W`.
    """

    vocab_size = len(vocab)
    
    # Matrix vectors
    W = "Inicializar esto"

    #biases
    biases = "inicializar esto"

    
    # Igual que antes, no implementaremos esta parte.
    
    gradient_squared = np.ones((vocab_size * 2, vector_size),
                               dtype=np.float64)

    # Sum of squared gradients for the bias terms.
    gradient_squared_biases = np.ones(vocab_size * 2, dtype=np.float64)

    data = [(W[i_main], W[i_context + vocab_size],
             biases[i_main : i_main + 1],
             biases[i_context + vocab_size : i_context + vocab_size + 1],
             gradient_squared[i_main], gradient_squared[i_context + vocab_size],
             gradient_squared_biases[i_main : i_main + 1],
             gradient_squared_biases[i_context + vocab_size
                                     : i_context + vocab_size + 1],
             cooccurrence)
            for i_main, i_context, cooccurrence in cooccurrences]

    for i in tqdm(range(iterations), desc='training'):
        
        cost = run_iter(vocab, data, **kwargs)
        if iter_callback is not None:
            iter_callback(W)

    return W

In [0]:
cooccurrences = build_cooccur(w2id, quijote_docs, vocab_size=config['vocab_size'],
                             window_size=5,
                             min_count=None)

In [0]:
from random import shuffle
import itertools
import logging

logger = logging.getLogger("glove")

In [0]:
config = {
    'vocab_size': len(vocab),
    'embed_size': 100,
    'context_size':3
}
logger = logging.getLogger("glove")

In [0]:
from sklearn.metrics.pairwise import cosine_similarity

k_close = 10